In [2]:
import os
import tarfile

import tensorflow as tf
from bob.learn.tensorflow.models.inception_resnet_v2 import InceptionResNetV2
from bob.learn.tensorflow.utils import restore_model_variables_from_checkpoint

In [3]:
# make sure you are using tensorflow 2
tf.__version__

'2.3.1'

In [4]:
checkpoint = "/tmp/msceleb/inception-v2_batchnorm_rgb"
path = f"{checkpoint}.tar.gz"
os.makedirs(checkpoint, exist_ok=True)
tf.keras.utils.get_file(
    path,
    "https://www.idiap.ch/software/bob/data/bob/bob.bio.face_ongoing/master/msceleb/inception-v2_batchnorm_rgb.tar.gz",
)
with tarfile.open(path) as f:
    f.extractall(os.path.dirname(checkpoint))

In [5]:
%%bash
# strip the folder names from saved checkpoint
sed -i "s|/.*/||" $checkpoint/checkpoint

In [6]:
# investigate the variable names from the checkpoint
# This model was saved using the following architecture code:
# https://gitlab.idiap.ch/bob/bob.learn.tensorflow/-/blob/3a6a2531c7978edfba09c601667694ab2ae2359e/bob/learn/tensorflow/network/InceptionResnetV2.py
tf.train.list_variables(tf.train.latest_checkpoint(checkpoint))

[('InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta', [192]),
 ('InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_mean', [192]),
 ('InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_variance',
  [192]),
 ('InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/weights', [1, 1, 2080, 192]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0a_1x1/BatchNorm/beta', [192]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean',
  [192]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance',
  [192]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0a_1x1/weights',
  [1, 1, 2080, 192]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0b_1x3/BatchNorm/beta', [224]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0b_1x3/BatchNorm/moving_mean',
  [224]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0b_1x3/BatchNorm/moving_variance',
  [224]),
 ('InceptionResnetV2/Block8/Branch_1/Conv2d_0b_1x3/weights', [1, 3, 192, 224]),
 ('InceptionResnetV2

In [7]:
# Create the model and look at its structure
# This is the model's code in tf2 format:
# https://gitlab.idiap.ch/bob/bob.learn.tensorflow/-/blob/ee013d0d47dbd43a447abcb7b665a7cd75398cce/bob/learn/tensorflow/models/inception_resnet_v2.py
model = InceptionResNetV2(input_shape=(160, 160, 3), include_top=False, bottleneck=True)
model.summary()

Model: "InceptionResnetV2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2d_1a_3x3 (SequentialLay (None, 79, 79, 32)        960       
_________________________________________________________________
Conv2d_2a_3x3 (SequentialLay (None, 77, 77, 32)        9312      
_________________________________________________________________
Conv2d_2b_3x3 (SequentialLay (None, 77, 77, 64)        18624     
_________________________________________________________________
MaxPool_3a_3x3 (MaxPooling2D (None, 38, 38, 64)        0         
_________________________________________________________________
Conv2d_3b_1x1 (SequentialLay (None, 38, 38, 80)        5360      
_________________________________________________________________
Conv2d_4a_3x3 (SequentialLay (None, 36, 36, 192)       138816    
_________________________________________________________________
MaxPool_5a_3x3 (MaxPooling2D (None, 17, 17, 192) 

In [8]:
# investigate the variable names in the model
[var.name for var in model.variables]

['InceptionResnetV2/Conv2d_1a_3x3/Conv2D/kernel:0',
 'InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/beta:0',
 'InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/moving_mean:0',
 'InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/moving_variance:0',
 'InceptionResnetV2/Conv2d_2a_3x3/Conv2D/kernel:0',
 'InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/beta:0',
 'InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/moving_mean:0',
 'InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/moving_variance:0',
 'InceptionResnetV2/Conv2d_2b_3x3/Conv2D/kernel:0',
 'InceptionResnetV2/Conv2d_2b_3x3/BatchNorm/beta:0',
 'InceptionResnetV2/Conv2d_2b_3x3/BatchNorm/moving_mean:0',
 'InceptionResnetV2/Conv2d_2b_3x3/BatchNorm/moving_variance:0',
 'InceptionResnetV2/Conv2d_3b_1x1/Conv2D/kernel:0',
 'InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/beta:0',
 'InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_mean:0',
 'InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_variance:0',
 'InceptionResnetV2/Conv2d_4a_3x3/Conv2D/kernel:0',
 'InceptionResnetV2/Conv2d_4a_3x

In [9]:
# Create a function that converts variable names of the model to checkpoint names
# If the mapped variable names are not found in the checkpoint, an error is raised
# To make this job easier, it's best to change the variables in the Keras model to
# match the names in the checkpoint and then do minor modifications here.
def normalizer(name):
    old_name = name = name.split(":")[0]

    # name differences between slim layer names and keras layer names
    name = name.replace("/kernel", "/weights")
    name = name.replace("/bias", "/biases")

    # for Conv2D layers
    name = name.replace("Conv2D/weights", "weights")
    name = name.replace("Conv2D/biases", "biases")

    # name differences between our architecture and the checkpoint
    name = name.replace("block35", "Repeat/block35")
    name = name.replace("block17", "Repeat_1/block17")
    name = name.replace("block8", "Repeat_2/block8")
    name = name.replace("Repeat_2/block8_10", "Block8")

    print(f"{old_name} -> {name}")
    return name


restore_model_variables_from_checkpoint(model, checkpoint, normalizer=normalizer)


InceptionResnetV2/Conv2d_1a_3x3/Conv2D/kernel -> InceptionResnetV2/Conv2d_1a_3x3/weights
InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/beta -> InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/beta
InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/moving_mean -> InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/moving_mean
InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/moving_variance -> InceptionResnetV2/Conv2d_1a_3x3/BatchNorm/moving_variance
InceptionResnetV2/Conv2d_2a_3x3/Conv2D/kernel -> InceptionResnetV2/Conv2d_2a_3x3/weights
InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/beta -> InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/beta
InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/moving_mean -> InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/moving_mean
InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/moving_variance -> InceptionResnetV2/Conv2d_2a_3x3/BatchNorm/moving_variance
InceptionResnetV2/Conv2d_2b_3x3/Conv2D/kernel -> InceptionResnetV2/Conv2d_2b_3x3/weights
InceptionResnetV2/Conv2d_2b_3x3/BatchNorm/beta -> InceptionResnetV2/Conv2d_2b_

In [10]:
# now the save the model in tensorflow 2 format
model.save(f"{checkpoint}_v2_format", save_format="tf")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /idiap/home/amohammadi/scratch/tmp/msceleb/inception-v2_batchnorm_rgb_v2_format/assets


In [14]:
# load the old model with a conda installation of old tensorflow
# create a random input, give it to the network, and save both input and output
# the code below is an exmple, but this cell must run in another notebook with an
# old bob (like bob 7) installation
assert (
    False
), "run the code in this cell in another notebook with a Bob 7 conda environment, do `conda create -n bob7 -c <bob-conda-channel> bob=7 bob.learn.tensorflow`"

import numpy as np
import tensorflow as tf
from bob.learn.tensorflow.network import inception_resnet_v2_batch_norm

input_shape = (1, 160, 160, 3)
input_tensor = tf.placeholder(tf.float32, shape=input_shape)

np.random.seed(10)
data = np.random.rand(1, 160, 160, 3).astype("float32")
np.save("input.npy", data)

for mode, name in [
    (tf.estimator.ModeKeys.TRAIN, "train_mode"),
    (tf.estimator.ModeKeys.PREDICT, "predict_mode"),
]:
    prelogits = inception_resnet_v2_batch_norm(input_tensor, mode=mode)[0]
    checkpoint = "/tmp/msceleb/inception-v2_batchnorm_rgb"
    session = tf.compat.v1.Session()
    session.run(tf.compat.v1.global_variables_initializer())
    saver = tf.compat.v1.train.Saver()
    saver.restore(session, tf.train.latest_checkpoint(checkpoint))
    output = session.run(prelogits, feed_dict={input_tensor: data})
    np.save(f"output_{name}.npy", output)

In [14]:
# now test the input on our new model and see if it matches the old output
import numpy as np

data = np.load("input.npy")
ref_predict = np.load("output_predict_mode.npy")
ref_train = np.load("output_train_mode.npy")
data.shape, data.dtype

((1, 160, 160, 3), dtype('float32'))

In [18]:
output_predict = model(data, training=False).numpy()
output_predict

array([[ 1.0397079 ,  0.9887184 , -1.0198569 , -0.715245  ,  1.2795925 ,
        -0.87935054,  0.78027606, -0.72593015, -2.5908837 ,  0.12451646,
        -0.902159  , -0.47172827, -0.18754381,  1.522578  , -2.1243768 ,
        -0.01743102, -0.7484883 ,  0.99135643, -0.20708205, -0.21483701,
         0.02057505,  1.129887  ,  1.3154736 , -0.6671123 ,  0.49072063,
        -1.5672749 , -0.89510536,  0.7542453 ,  0.9450621 , -1.4345936 ,
         0.7585461 , -0.07696939, -2.1004128 , -0.49109945, -1.1812558 ,
        -1.1648568 , -0.87621725,  0.8581683 ,  0.7901715 , -1.0273908 ,
         0.7262468 ,  0.9869791 , -2.0192435 ,  1.1500988 ,  1.0887905 ,
         2.6515627 ,  0.00478584, -1.6734318 , -0.01522779, -0.7636968 ,
         2.0314407 , -0.7928898 ,  0.45450714, -2.1267045 , -0.4153372 ,
         0.62672305,  0.10927075,  0.41156793, -1.0595654 ,  0.3506987 ,
         0.20369251,  0.39730582,  1.7632729 ,  0.57014084,  0.08623758,
         0.5078342 , -0.28096065,  1.9077072 , -1.2

In [16]:
# they are close enough
np.testing.assert_allclose(output_predict, ref_predict, rtol=1e-05)

AssertionError: 
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 12 / 128 (9.38%)
Max absolute difference: 1.66893e-06
Max relative difference: 9.847395e-05
 x: array([[ 1.039708,  0.988718, -1.019857, -0.715245,  1.279593, -0.879351,
         0.780276, -0.72593 , -2.590884,  0.124516, -0.902159, -0.471728,
        -0.187544,  1.522578, -2.124377, -0.017431, -0.748488,  0.991356,...
 y: array([[ 1.039708,  0.988719, -1.019857, -0.715245,  1.279593, -0.879351,
         0.780276, -0.725931, -2.590884,  0.124517, -0.902159, -0.471728,
        -0.187544,  1.522577, -2.124377, -0.017431, -0.748489,  0.991358,...

In [17]:
# they are close enough
np.testing.assert_allclose(output_predict, ref_predict, rtol=1e-04)

In [20]:
# test in train mode
output_train = model(data, training=True).numpy()
output_train

array([[ 1.2059948 ,  0.90457535, -1.6966419 , -0.6469312 ,  1.8390388 ,
        -0.82886887,  0.20409298, -1.2975912 , -2.2515354 ,  0.16945124,
        -0.6430161 , -0.7341013 , -0.37252808,  1.208333  , -2.1846251 ,
         0.5799656 , -0.9761491 ,  0.5671108 , -0.46018916, -1.0497742 ,
         0.36604214,  1.0983484 ,  1.6091515 , -0.4587916 ,  0.49420023,
        -1.3356848 , -0.7289448 ,  0.98772526,  1.3689709 , -1.3146491 ,
         0.69885445, -0.82313013, -1.8048697 , -0.0885191 , -0.95188236,
        -1.2162575 , -0.20026398,  0.84856796,  0.2631035 , -1.0577492 ,
         0.42345095,  1.2333517 , -1.5610876 , -0.29854918,  0.58075297,
         0.9959574 ,  0.18556404, -1.828722  , -0.43610954, -0.6100631 ,
         1.5355668 , -0.88698196, -0.24892426, -1.5401478 , -0.20694387,
        -0.43096346,  0.34370852,  0.42531586, -1.1192467 , -0.26605558,
         0.32238102, -0.09467888,  1.9468937 ,  0.1408875 ,  0.33964324,
         0.47477245, -0.40066433,  1.738759  , -1.5

In [21]:
# they are close enough
np.testing.assert_allclose(output_train, ref_train, rtol=1e-05)

AssertionError: 
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 1 / 128 (0.781%)
Max absolute difference: 7.748604e-07
Max relative difference: 4.5886292e-05
 x: array([[ 1.205995,  0.904575, -1.696642, -0.646931,  1.839039, -0.828869,
         0.204093, -1.297591, -2.251535,  0.169451, -0.643016, -0.734101,
        -0.372528,  1.208333, -2.184625,  0.579966, -0.976149,  0.567111,...
 y: array([[ 1.205995,  0.904575, -1.696642, -0.646931,  1.839038, -0.828869,
         0.204093, -1.297591, -2.251535,  0.169451, -0.643016, -0.734101,
        -0.372528,  1.208333, -2.184625,  0.579965, -0.976149,  0.567111,...

In [23]:
# they are close enough
np.testing.assert_allclose(output_train, ref_train, rtol=1e-04)